In [1]:
from OpenInterface.api_calls import generate_response
import json
from Models.character import Character
from Models.chapter import Chapter
from Models.scene import Scene
from Models.story import Story
import Prompts

story = Story()

In [2]:
def export_book():
    # Create a list to store all the scene texts
    all_text_output = []
    all_text_output.append("This is the beginning of the book.")
    all_text_output.append("Summary:")
    all_text_output.append(story.summary)
    all_text_output.append("\n\n")

    all_text_output.append("Acts:")
    for act in story.acts:
        all_text_output.append(act)
        all_text_output.append("\n\n")

    all_text_output.append("Plotlines:")
    for plotline in story.plotlines:
        all_text_output.append(plotline)
        all_text_output.append("\n\n")

    all_text_output.append("Characters:")        
    for character in story.characters:
        all_text_output.append(character.name)
        all_text_output.append("\n")
        all_text_output.append(str(character.__dict__))

    for chapter in story.chapters:
        all_text_output.append(f"CHAPTER: {chapter.id}\n" + chapter.name)
        all_text_output.append(chapter.description)
        all_text_output.append("\n")

        all_text_output.append("Goals:")
        all_text_output.append(chapter.goals)
        all_text_output.append("\n")

        all_text_output.append(f"SCENES: {len(chapter.scenes)}")
        for scene in chapter.scenes:
            all_text_output.append(f"Scene {scene.id}: " + scene.name)
            all_text_output.append("Description:")
            all_text_output.append(scene.description)
            all_text_output.append("Goals:")
            all_text_output.append(scene.goals)
            all_text_output.append(f"Act: {scene.act}")
            all_text_output.append(f"Chapter: {scene.chapter}")
            all_text_output.append(f"Plotpoint: {scene.plotpoint}")    
            all_text_output.append(f"Text: {scene.final_text}")

        all_text_output.append("----------------------")
        all_text_output.append("\n")

    # Write the scene texts to a text file
    output_file = "books/book.txt"
    with open(output_file, "w") as f:
        for scene_text in all_text_output:
            f.write(scene_text + "\n")

    print(f"Scene texts written to {output_file}")


# 1- User writes a One-Paragraph Summary

In [3]:
story.summary = "In the blood-soaked realm of “Sword and Sinew,” where treachery festers like a poisoned chalice, Lady Seraphina, a rogue sorceress with a penchant for vengeance, embarks on a perilous quest. Her mission: to retrieve the Crown of Shadows, an artifact said to grant dominion over life and death. But standing in her way are rival warlords, a duplicitous court jester, and a band of undead knights who hunger for her soul. As the moon waxes and secrets unravel, Lady Seraphina dances on the edge of damnation, wielding her blade and dark magic in a deadly waltz toward destiny."

# 2- Character Summaries: Create brief character descriptions.

In [4]:
def create_character_objects(characterData):
    """
    Converts a dictionary of character data into an array of Character objects.

    Args:
        characterData (dict): A dictionary containing character data.

    Returns:
        list: List of Character objects.
    """
    characters = []
    for char_data in characterData['characters']:
        char_obj = Character()
        char_obj.name = char_data.get('Name', '')
        char_obj.role = char_data.get('Role', '')
        char_obj.age = char_data.get('Age', '')
        char_obj.occupation = char_data.get('Occupation', '')
        char_obj.appearance = char_data.get('Appearance', '')
        char_obj.personality_traits = char_data.get('Personality Traits', '')
        char_obj.motivation = char_data.get('Motivation', '')
        char_obj.backstory = char_data.get('Backstory', '')
        characters.append(char_obj)
    return characters

In [5]:
characters = []

## WE COULD HAVE CHARACTER ROLES HERE: ANTAGONISTS; PROTAGONISTS; ETC.
## THIS COULD RUN 2 TIMES; SO WE HAVE EXTRA CHARACTERS.
## TODO: fix the output of this prompt to make it error free when decoding
prompt = Prompts.createCharacterFrom(story.summary)

response = generate_response(prompt)

charactersData = json.loads(response)
decodedCharacters = create_character_objects(charactersData)
characters += decodedCharacters

# # We save the character ids in the story object as well
for character in characters:
    story.characters.append(character)

AttributeError: module 'prompts' has no attribute 'createCharacterFrom'

# 3- Character Charts: Develop detailed character profiles.

In [49]:
def get_character_profile(character, summary):
    prompt = Prompts.generateCharacterProfile(character, summary)

    responseData = generate_response(prompt)
    return responseData

def get_character_keyScenes(character, summary):
    prompt = Prompts.generateCharacterKeyScenes(character, summary)

    responseData = generate_response(prompt)
    return responseData

# Iterate through all characters and update their profiles
for character in characters:
    char = character.to_dict()
    character.detailedProfile = get_character_profile(char, story.summary)
    character.keyScenes = get_character_keyScenes(char, story.summary)

KeyboardInterrupt: 

# 4- Plot Summary: Outline the major plot points.

In [10]:
char_dicts = []
for character in characters:
    char_dicts.append(character.to_dict())

prompt = Prompts.generateStoryAct(story.summary, char_dicts)

storyActs = generate_response(prompt).split('\n\n')

story.acts = storyActs

# 5- Create chapters out of the Acts


In [26]:
def generate_chapter(act, summary):
    prompt = Prompts.generateChapterFor(act, summary)

    story_chapters = generate_response(prompt).split('\n\n')

    print(story_chapters)

    for response in story_chapters:
        chapter = Chapter()
        chapter.description = response
        story.chapters.append(chapter)

story.chapters = []

for act in story.acts:
    generate_chapter(act, story.summary)

['Chapter 1: Lady Seraphina, armed with dark magic and a deadly blade, crosses the Threshold into the blood-soaked realm of "Sword and Sinew." She faces her first challenge in the form of the rival warlord Lord Aric, who also seeks the Crown of Shadows for his nefarious purposes. Their clash sets the stage for a dangerous game of cat and mouse as Lady Seraphina navigates the treacherous landscape filled with betrayal and ambushes.', "Chapter 2: Lady Seraphina encounters the duplicitous court jester Malvolio, who schemes to manipulate her for his own gain. Torn between trusting him and seeing through his deception, Lady Seraphina must tread carefully as she uncovers more layers of treachery and deceit surrounding the Crown of Shadows. Malvolio's motives remain unclear as they dance around each other in a dangerous game of manipulation.", 'Chapter 3: The tension between Lady Seraphina and Lord Aric reaches its peak as they engage in a deadly battle for control of the Crown of Shadows. Wi

### Expanding chapter description to other elements

In [28]:
for chapter in story.chapters:
    prompt = Prompts.generateChapterTitle(chapter, story.summary)

    chapter.name = generate_response(prompt)

    prompt = Prompts.generateChapterGoals(chapter, story.summary)

    chapter.goals = generate_response(prompt)

    prompt = Prompts.generateChapterConsequences(chapter, story.summary)

    chapter.consequences = generate_response(prompt)

    prompt = Prompts.generateChapterEndResult(chapter, story.summary)

    chapter.end_result = generate_response(prompt)

    prompt = Prompts.generateChapterCharacters(chapter, story.summary)

    chapter.characters = generate_response(prompt).strip("[]").split(", ")

# 6- Scene List: Create a list of scenes in the novel and write them

In [45]:
def create_scenes_from_chapter(chapter):
    prompt = Prompts.generateScenesFromChapter(chapter, chapter.act, story.summary)

    return generate_response(prompt).split(sep='\n\n')

In [46]:
def define_scene_details(chapter, scene):
    prompt = Prompts.generateSceneDetails(chapter, scene, story.summary)
    
    scene.name = generate_response(prompt)

    prompt = Prompts.generateSceneDescription(chapter, scene, story.summary)
    
    scene.goals = generate_response(prompt)

    prompt = Prompts.generateSceneConsequences(chapter, scene, story.summary)
    
    scene.consequences = generate_response(prompt)

    prompt = Prompts.generateSceneEndResult(scene, story.summary, chapter)
    
    scene.end_result = generate_response(prompt)

    prompt = Prompts.generateSceneStoryline(scene, story.summary, chapter)
    
    scene.final_text = generate_response(prompt)

In [51]:
previous_scene = ""
for chapter in story.chapters:
    print(f"Starting chapter {chapter.id}-{chapter.name}")
    sceneResponses = create_scenes_from_chapter(chapter)
    for sceneResponse in sceneResponses:
        scene = Scene()
        scene.description = sceneResponse
        scene.act = chapter.act
        scene.plotpoint = chapter.plotpoint
        scene.previous_scene = previous_scene
        scene.goals = chapter.goals
        scene.consequences = chapter.consequences
        scene.chapter = chapter.name
        define_scene_details(scene, chapter)
        chapter.scenes.append(scene)
        print(f"Creating scene {scene.id}-{scene.name}")
        print(f"Final Scene: {scene.__dict__}")

        previous_scene = "Before this scene, this happened: " + scene.description + scene.consequences

Starting chapter "Dancing on the Edge of Destiny"
Creating scene Scene 1: Lady Seraphina, cloaked in shadows and armed with a deadly blade, stealthily infiltrates the warlord Lord Aric's fortress, seeking the first piece of the puzzle that will lead her to the Crown of Shadows. As she navigates the dimly lit corridors, her goal is to find the elusive map rumored to be in Lord Aric's possession, but she must remain undetected to avoid the dire consequences if caught in enemy territory. The end result of this scene sees Lady Seraphina successfully locating the map, a crucial step towards her ultimate goal, but the risk of discovery looms over her as she prepares to make her escape.
Creating scene Scene 2: In a tense showdown in the courtyard of the fortress, Lady Seraphina comes face to face with Lord Aric, the rival warlord who seeks the Crown of Shadows for his own dark purposes. Their clash of wills and blades sets the stage for a high-stakes game of cat and mouse, each trying to outm

# 9- Write the Novel: How can we make the final_text be longer, bigger and uncut?

In [ ]:
# chapters = []

# for scene in scenes:
#     prompt = f"""
#         Considering the summary of this scene {scene}, which belongs to this act of the story {act}, and the characters {characters}, write the scene.
#     """

#     chapters.append(api_calls(prompt))

# 10- Join all the chapters into a txt and export it.

In [48]:
export_book()

Scene texts written to book.txt
